In [99]:
import pandas as pd
import numpy as np
import ast
import mne
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

**Load Data**

In [100]:
processed_path = '../Data/Processed/sub01_binned.csv'
eeg_data = pd.read_csv(processed_path)

**Random Forest Model for Each Electrode**

In [134]:
def rf_each_electrode(eeg_data, n_estimators=100):
    electrode_data = eeg_data.drop(columns=['label', 'label_type'])
    y = eeg_data['label_type']
    electrode_accuracies = {}

    for electrode in tqdm(electrode_data.columns, desc='Training Data'):
        X_electrode = electrode_data[electrode].apply(lambda x: ast.literal_eval(x)).tolist()
        X_electrode = pd.DataFrame(X_electrode)

        X_train, X_test, y_train, y_test = train_test_split(X_electrode, y, test_size=0.2, random_state=42)

        rf = RandomForestClassifier(n_estimators=n_estimators, random_state=42)
        rf.fit(X_train, y_train)

        y_pred = rf.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        electrode_accuracies[electrode] = accuracy
    
    return electrode_accuracies

In [ ]:
electrode_accuracies = rf_each_electrode(eeg_data, n_estimators=200)

In [111]:
sorted_electrodes = sorted(electrode_accuracies.items(), key=lambda item: item[1], reverse=True)
top_electrodes = sorted_electrodes[:10]

In [112]:
for electrode, accuracy in top_electrodes:
    print(f"{electrode}: {accuracy:.4f}")

CP3: 0.6250
PO3: 0.6250
C3: 0.6094
C5: 0.5938
TP7: 0.5938
PO7: 0.5781
T7: 0.5781
CP1: 0.5781
C2: 0.5781
P5: 0.5625


**Random Forest with Mean**

In [140]:
def rf_w_mean(eeg_data, n_estimators=100):
    electrode_data = eeg_data.drop(columns=['label', 'label_type'])
    y = eeg_data['label_type']

    mean_data = electrode_data.map(lambda x: np.mean(ast.literal_eval(x)) if isinstance(x, str) else np.nan)

    X = mean_data.values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    rf_mean = RandomForestClassifier(n_estimators=n_estimators, random_state=42)
    rf_mean.fit(X_train, y_train)

    y_pred = rf_mean.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    print(f"Random Forest with Mean Accuracy: {accuracy:.4f}")

In [132]:
rf_w_mean(eeg_data)

Random Forest with Mean Accuracy: 0.3750


**Random Forest During Task**

In [135]:
eeg_task_path = '../Data/Processed/sub01_binned_task.csv'
eeg_task = pd.read_csv(eeg_task_path)

task_accuracies = rf_each_electrode(eeg_task, n_estimators=200)

Training Data: 100%|██████████| 64/64 [01:36<00:00,  1.51s/it]


In [136]:
sorted_electrodes = sorted(task_accuracies.items(), key=lambda item: item[1], reverse=True)
top_electrodes = sorted_electrodes[:10]

for electrode, accuracy in top_electrodes:
    print(f"{electrode}: {accuracy:.4f}")

P5: 0.6875
T8: 0.6562
C1: 0.6562
FC6: 0.6406
PO7: 0.6250
Pz: 0.6250
Fz: 0.6094
P9: 0.6094
P2: 0.6094
Iz: 0.6094


**Random Forest with Mean During Task**

In [142]:
rf_w_mean(eeg_task, n_estimators=200)

Random Forest with Mean Accuracy: 0.6250


**Convolutional Neural Network**